# Prediction based on CaDRReS-Sc pre-trained model
This notebook show an example of how load a pre-trained CaDRReS-SC model and predict drug response based on new data.

In [1]:
import sys, os, pickle
from collections import Counter
import importlib
from ipywidgets import widgets
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

scriptpath = '..'
sys.path.append(os.path.abspath(scriptpath))

from cadrres_sc import pp, model, evaluation, utility

# Read pre-trained model

In [2]:
model_dir = '../example_result/'

In [3]:
obj_function = widgets.Dropdown(options=['cadrres-wo-sample-bias', 'cadrres-wo-sample-bias-weight'], description='Objetice function')

In [4]:
#choose which model you have trained previously
display(obj_function)

Dropdown(description='Objetice function', options=('cadrres-wo-sample-bias', 'cadrres-wo-sample-bias-weight'),…

## Load the pre-trained model based on your selection


In [5]:
model_spec_name = obj_function.value
model_file = model_dir + '{}_param_dict.pickle'.format(model_spec_name)

cadrres_model = model.load_model(model_file)

# Read test data
Again, for this example we load GDSC dataset.
@TODO: GDSC dataset using only essential gene list?

Note: GDSC_exp.tsv can be downloaded from https://www.dropbox.com/s/3v576mspw5yewbm/GDSC_exp.tsv?dl=0

## Notes for other test data

You can apply the model to other gene expression dataset. The input gene expression matrix should have been normalized, i.e. **for each sample, expression values are comparable across genes**. 

In this example the gene expression matrix provided by GDSC is already normalized using RMA.

For RNA-seq data, read count should be normalized by gene length, using normalization methods such as TPM.

In [6]:
gene_exp_df = pd.read_csv('../data/GDSC/GDSC_exp.tsv', sep='\t', index_col=0)
gene_exp_df = gene_exp_df.groupby(gene_exp_df.index).mean()
print("Dataframe shape:", gene_exp_df.shape, "\n")
gene_exp_df.head(2)

Dataframe shape: (17419, 1018) 



,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,907045,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954.1
GENE,,,,,,,,,,,,,,,,,,,,,
A1BG,6.208447,5.025810,5.506955,4.208349,3.399366,4.917872,3.828088,5.146903,3.107543,5.062066,...,4.272172,3.435025,4.930052,2.900213,4.523712,5.074951,2.957153,3.089628,4.047364,5.329524
A1CF,2.981775,2.947547,2.872071,3.075478,2.853231,3.221491,2.996355,2.893977,2.755668,2.985650,...,2.941659,3.155536,2.983619,3.118312,2.975409,2.905804,2.944488,2.780003,2.870819,2.926353


## Calculate fold-change
We normalized baseline gene expression values for each gene by computing fold-changes compared to the median value across cell-lines

In [7]:
cell_line_log2_mean_fc_exp_df, cell_line_mean_exp_df = pp.gexp.normalize_log2_mean_fc(gene_exp_df)

## Read essential genes list

Or in case you want your training using one specific set of genes.

In [8]:
ess_gene_list = utility.get_gene_list('../data/essential_genes.txt')

## Calculate kernel feature 

Based on all given cell line samples with gene expression profiles and a list of genes (e.g. essential gene list). This step might take a bit more time than usual.

In [ ]:
test_kernel_df = pp.gexp.calculate_kernel_feature(cell_line_log2_mean_fc_exp_df, cell_line_log2_mean_fc_exp_df, ess_gene_list)

Calculating kernel features based on 1610 common genes
(17419, 1018) (17419, 1018)
100 of 1018 (8.24)s
200 of 1018 (8.23)s
300 of 1018 (8.25)s
400 of 1018 (8.18)s
500 of 1018 (8.16)s


In [ ]:
print("Dataframe shape:", test_kernel_df.shape, "\n")
test_kernel_df.head(2)

# Drug response prediction

In [ ]:
print('Predicting drug response using CaDRReS: {}'.format(model_spec_name))
pred_df, P_test_df= model.predict_from_model(cadrres_model, test_kernel_df, model_spec_name)
print('done!')

Inspecting the model predictions and save the predictions

In [ ]:
pred_df.head()

In [ ]:
P_test_df.head()

In [ ]:
print('Saving ' + model_dir + '{}_test_pred.csv'.format(model_spec_name))
pred_df.to_csv(model_dir + '{}_test_pred.csv'.format(model_spec_name))

---

**Authors:** [Chayaporn Suphavilai](mailto:@.com), [Rafael Peres da Silva](), Genome Institute of Singapore, Nagarajan Lab, November, 2020

---

Reproducibility tips from https://github.com/jupyter-guide/ten-rules-jupyter